In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
import sys

In [2]:

import torch
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pytorch_lightning as pl
import wandb

import time
import nvtx

from loss import res_kp_loss
from utility import *

class KeypointNetModule(pl.LightningModule):
    def __init__(self, config, wandb_run, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters("learning_rate")
        self.config = config    
        #print("Pose ResNet is on device " + str(next(self.pose_hrnet.parameters()).get_device()))     # testing line
        #print("Is Pose ResNet on GPU? " + str(next(self.pose_hrnet.parameters()).is_cuda))            # testing line
        self.wandb_run = wandb_run
        self.loss_fn = res_kp_loss(
            gaussian_amp=self.config.dataset['GAUSSIAN_AMP'],
            gaussian_sigma=self.config.dataset['GAUSSIAN_STDDEV']
            )

        # ! Good grief what a hack. This is only for local testing.
        #WRITE_CSV = '/home/sasank/Documents/GitRepos/PnP-Solver/kp_estimates/naive_Ten_Dogs_64KP_estimates_pr.csv'
        #self.csv_file = pd.read_csv(WRITE_CSV)
        
        self.in_channels = self.config.dataset['IMG_CHANNELS']
        self.num_keypoints = self.config.dataset['NUM_KEY_POINTS']
        self.batch_size = self.config.datamodule['BATCH_SIZE']      # This is used in the logging steps in validation_step and test_step.
        self.image_height = self.config.dataset['IMAGE_HEIGHT']     # These two, too.
        self.image_width = self.config.dataset['IMAGE_WIDTH']

        self.my_dict = nn.ModuleDict({})

        self.my_dict["pre_block"] = nn.Sequential(
            nn.Conv2d(self.in_channels, 3, kernel_size=1, stride=1, padding=0, bias=False),     # This is just to convert the input channels to 3, which is what the resnet expects.
            nn.BatchNorm2d(3),
            nn.ReLU()
        )
        
        #self.my_dict["resnet"] = make_resnet(3, 64, 3, 1)
        # this 
        #self.my_dict["resnet"] = torchvision.models.resnet34(weights='IMAGENET1K_V1')
        # ! Trying out Resnet152 to see if it improves model output
        self.my_dict["resnet"] = torchvision.models.resnet152(weights='IMAGENET1K_V2')

        #assert self.num_keypoints <= 50, "If num_keypoints > 50, the last linear layer is growing bigger, which seems unreasonable."
        # The above assertion does not need to be made because it could be that the model is just finding a lower-dimensional representation of the keypoints, which, if accurate, would be a good thing.
        # ! If model performance is bad is bad when using 64 keypoints, then it may be a good idea to reexamine this.

        self.my_dict["keypoints"] = nn.Sequential(
            nn.Linear(1000, 500),
            nn.ReLU(),
            nn.Linear(500, 250),
            nn.ReLU(),
            nn.Linear(250, 2 * self.num_keypoints)
        )

    def forward(self, x):
        """This performs a forward pass on the dataset.

        Args:
            x (torch.Tensor): This is a tensor containing the input data.

        Returns:
            the forward pass of the dataset.
        """
        x = self.my_dict["pre_block"](x)
        x = self.my_dict["resnet"](x)
        x = self.my_dict["keypoints"](x)
        return x

    def configure_optimizers(self):
        #optimizer = torch.optim.Adam(self.parameters, lr=1e-3)
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        return optimizer

    @nvtx.annotate("Training step", color="red", domain="my_domain")
    def training_step(self, train_batch, batch_idx):
        training_batch, training_batch_labels = train_batch['image'], train_batch['kp_label']
        x = training_batch
        training_output = self(x)
        loss = self.loss_fn(training_output, training_batch_labels)
        #self.wandb_run.log('train/loss', loss, on_step=True)
        #print("Training loss: " + str(loss.item()))        # This print statement messes ups the pytorch_lightning progress bar lol 
        self.wandb_run.log({'train/loss': loss.item()})
        #print("First outputs and labels " + str(training_output[0]) + " " + str(training_batch_labels[0].item()))
        #self.log(name="train/loss", value=loss)
        return loss

    @nvtx.annotate("Validation step", color="green", domain="my_domain")
    def validation_step(self, validation_batch, batch_idx):
        val_batch, val_batch_labels = validation_batch['image'], validation_batch['kp_label']
        full_val_batch = validation_batch['full_image']     # 'Full' here means the images without subset_pixel applied
        img_names = validation_batch['img_name']
        x = val_batch
        val_output = self(x)
        loss = self.loss_fn(val_output, val_batch_labels)
        #print("Validation loss: " + str(loss.item()))      # This print statement messes ups the pytorch_lightning progress bar lol
        self.wandb_run.log({'validation/loss': loss.item()})


        # * Logging the predictions
        # Must remember that val_output is a tensor of shape (batch_size, 2 * num_keypoints)
        # And x is a tensor of shape (batch_size, 1, self.image_height, self.image_width)
        """
        fig_output = plot_val_images(images=full_val_batch, preds=val_output, labels=val_batch_labels, img_names=img_names, num_keypoints=self.num_keypoints, title='Unsubsetted Image')
        self.wandb_run.log({f'validation/val_output_{batch_idx}': fig_output})
        # Plot the model output from what the model actually sees
        fig_subsetted_output = plot_val_images(images=val_batch, preds=val_output, labels=val_batch_labels, img_names=img_names, num_keypoints=self.num_keypoints, title='Model View')
        self.wandb_run.log({f'validation/epoch_{str(self.current_epoch)}_val_subsetted_output_{batch_idx}': fig_subsetted_output})
        # Just plot the input images
        fig_input = plot_inputs(images=full_val_batch, img_names=img_names, title='Input Image')
        self.wandb_run.log({f'validation/epoch_{str(self.current_epoch)}_val_input_{batch_idx}': fig_input})
        """

        # Use plot_test_images to plot the images
        # ! Disabling val plotting for now to see how the models do
        """
        fig_output_vector = plot_outputs(images=full_val_batch, preds=val_output, labels=val_batch_labels, img_names=img_names, num_keypoints=self.num_keypoints, title='Unsubsetted Image')
        fig_subsetted_output_vector = plot_outputs(images=val_batch, preds=val_output, labels=val_batch_labels, img_names=img_names, num_keypoints=self.num_keypoints, title='Model View')
        fig_intput_vector = plot_inputs(images=full_val_batch, img_names=img_names, title='Input Image')

        for i in range(len(fig_output_vector)):
            self.wandb_run.log({f'validation/{img_names[i]}/E{self.current_epoch}_full_output': fig_output_vector[i]})
            self.wandb_run.log({f'validation/{img_names[i]}/E{self.current_epoch}_subsetted_output': fig_subsetted_output_vector[i]})
            self.wandb_run.log({f'validation/{img_names[i]}/E{self.current_epoch}_input': fig_intput_vector[i]})
        """

        return loss



    @nvtx.annotate("Test step", color="blue", domain="my_domain")
    def test_step(self, test_batch, batch_idx):
        input_test_batch, test_batch_labels = test_batch['image'], test_batch['kp_label']          # 'input' here means the images inputted to the model, often with subset_pixel applied
        full_test_batch = test_batch['full_image']     # 'Full' here means the images without subset_pixel applied
        img_names = test_batch['img_name']
        x = input_test_batch
        test_output = self(x)
        loss = self.loss_fn(test_output, test_batch_labels)

        """
        for i in range(len(test_output)):
            # We will plot the outputs, which are 2D keypoints, to self.csv_file in the row that corresponds to the image name
            keypoints = test_output[i].detach().cpu().numpy()
            # Make the keypoints a 2D array of shape (num_keypoints, 2)
            keypoints = keypoints.reshape(self.num_keypoints, 2)
            #keypoints.view(64,2)
            img_name = img_names[i]

            # Find the row that corresponds to the image name
            #self.csv_file.loc[self.csv_file['Image address'] == img_name]['Femur PR KP points'] = str(keypoints)
            self.csv_file.loc[self.csv_file['Image address'] == img_name, 'Femur PR KP points'] = str(keypoints)
        """

        # Logging the predictions
        # Must remember that test_output is a tensor of shape (batch_size, 2 * num_keypoints)
        # And x is a tensor of shape (batch_size, 1, self.image_height, self.image_width)
        data_set_name = 'naive_set' if self.config.datamodule['USE_NAIVE_TEST_SET'] else 'test_set'
        fig_output_vector = plot_outputs(images=full_test_batch, preds=test_output, labels=test_batch_labels, img_names=img_names, num_keypoints=self.num_keypoints, title='Unsubsetted Image')
        fig_subsetted_output_vector = plot_outputs(images=input_test_batch, preds=test_output, labels=test_batch_labels, img_names=img_names, num_keypoints=self.num_keypoints, title='Model View')
        fig_input_vector = plot_inputs(images=full_test_batch, img_names=img_names, title='Input Image')

        for i in range(len(fig_output_vector)):
            self.wandb_run.log({f'test/{data_set_name}/{img_names[i]}/full_output': fig_output_vector[i]})
            self.wandb_run.log({f'test/{data_set_name}/{img_names[i]}/subsetted_output': fig_subsetted_output_vector[i]})
            self.wandb_run.log({f'test/{data_set_name}/{img_names[i]}/input': fig_input_vector[i]})



        return loss
    

/home/sasank/miniconda3/envs/jtml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import torch.nn as nn
import albumentations as A
import numpy as np
import time
import os

"""
ResNet-based Keypoint Estimator
"""
class Fem_Configuration:
    def __init__(self):
        """
        self.temp = {
            'train_data': '/home/sasank/Documents/GitRepos/Sasank_JTML_seg/data/3_2_22_fem/train_3_2_22_fem.csv',
            'val_data': '/home/sasank/Documents/GitRepos/Sasank_JTML_seg/data/3_2_22_fem/val_3_2_22_fem.csv',
            'test_data': '/home/sasank/Documents/GitRepos/Sasank_JTML_seg/data/3_2_22_fem/test_3_2_22_fem.csv'
        }
        """
        self.init = {
            'PROJECT_NAME': 'Keypoint Estimation',
            'MODEL_NAME': 'Fem_64KP',
            'RUN_NAME': time.strftime('%Y-%m-%d-%H-%M-%S'),
            'WANDB_RUN_GROUP': 'Local',
            'FAST_DEV_RUN': False,  # Runs inputted batches (True->1) and disables logging and some callbacks
            'MAX_EPOCHS': 10,
            'MAX_STEPS': -1,    # -1 means it will do all steps and be limited by epochs
            'STRATEGY': None    # This is the training strategy. Should be 'ddp' for multi-GPU (like HPG)
        }
        self.etl = {
            'RAW_DATA_FILE': -1,
            'DATA_DIR': "data",
            # Lol what is this?
            'KEYPOINT_DIRECTORY': "keypoints",
            'KEYPOINT_TXT_FILES': ['tib_KPlabels_16.txt'],
            'VAL_SIZE':  0.2,       # looks sus
            'TEST_SIZE': 0.01,      # I'm not sure these two mean what we think
            #'random_state': np.random.randint(1,50)
            # HHG2TG lol; deterministic to aid reproducibility
            'RANDOM_STATE': 42,

            'CUSTOM_TEST_SET': False,
            'TEST_SET_NAME': '/my/test/set.csv'
        }

        self.dataset = {
            'DATA_NAME': 'Ten_Dogs_64KP',
            'SUBSET_PIXELS': True,
            'IMAGE_HEIGHT': 1024,
            'IMAGE_WIDTH': 1024,
            'MODEL_TYPE': 'fem',        # how should we do this? not clear this is still best...
            'CLASS_LABELS': {0: 'bone', 1: 'background'},
            'NUM_KEY_POINTS': 64,
            'IMG_CHANNELS': 1,      # Is this different from self.module['NUM_IMAGE_CHANNELS']
            'STORE_DATA_RAM': False,
            'IMAGE_THRESHOLD': 0,
            'USE_ALBUMENTATIONS': False,

            # What do these do?
            'NUM_PRINT_IMG' : 1,
            'KP_PLOT_RAD' : 3,

            #'NUM_POINTS' : 128,

            'GAUSSIAN_STDDEV' : 5,
            'GAUSSIAN_AMP' : 1e3,

            'STORE_DATA_RAM' : False,

            'CROP_IMAGES' : False,
            'CROP_MIN_X' : 0.29,
            'CROP_MAX_X' : 0.84,
            'CROP_MIN_Y' : 0.45,
            'CROP_MAX_Y' : 0.95,
            
            'IMAGES_PER_GRID': 1,
            'per_grid_image_count_height' : 1, 
            'per_grid_image_count_width' : 1
        }

        """
        # segmentation_net_module needs to be below dataset because it uses dataset['IMG_CHANNELS']
        self.keypoint_net_module = {
            'NUM_KEY_POINTS': 128,
            'NUM_IMG_CHANNELS': self.dataset['IMG_CHANNELS']
        }
        """

        self.datamodule = {
            'IMAGE_DIRECTORY': '/media/sasank/LinuxStorage/Dropbox (UFL)/Canine Kinematics Data/TPLO_Ten_Dogs_grids/',
            'CKPT_FILE': None,
            'USE_NAIVE_TEST_SET': False,
            'BATCH_SIZE': 2,
            'SHUFFLE': True,        # Only for training; for test and val this is set in the datamodule script to False
            'NUM_WORKERS': 2,
            'PIN_MEMORY': False
            #'SUBSET_PIXELS': True - this is now in dataset
        }


        # hyperparameters for training
        self.hparams = {
            'LOAD_FROM_CHECKPOINT': False,
            'learning_rate': 1e-3
        }

        #self.transform = None
        self.transform = \
        A.Compose([
            # Let's do only rigid transformations for now
            A.HorizontalFlip(p=0.9),
            A.VerticalFlip(p=0.9),
            A.RandomRotate90(p=0.9),
            A.Transpose(p=0.9),
            #A.RandomGamma(always_apply=False, p = 0.5,gamma_limit=(10,300)),
            #A.ShiftScaleRotate(always_apply = False, p = 0.5,shift_limit=(-0.06, 0.06), scale_limit=(-0.1, 0.1), rotate_limit=(-180,180), interpolation=0, border_mode=0, value=(0, 0, 0)),
            #A.Blur(always_apply=False, blur_limit=(3, 10), p=0.2),
            #A.Flip(always_apply=False, p=0.5),
            #A.InvertImg(always_apply=False, p=0.5),
            #A.MultiplicativeNoise(always_apply=False, p=0.25, multiplier=(0.1, 2), per_channel=True, elementwise=True)
            #A.ElasticTransform(always_apply=False, p=0.85, alpha=0.5, sigma=150, alpha_affine=50.0, interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None, approximate=False),
            #A.CoarseDropout(always_apply = False, p = 0.25, min_holes = 1, max_holes = 100, min_height = 25, max_height=25),
        ],
        p=0.85)

In [4]:
import torch
import torch.nn as nn
import albumentations as A
import numpy as np
import time
import os

"""
ResNet-based Keypoint Estimator
"""
class Tib_Configuration:
    def __init__(self):
        """
        self.temp = {
            'train_data': '/home/sasank/Documents/GitRepos/Sasank_JTML_seg/data/3_2_22_fem/train_3_2_22_fem.csv',
            'val_data': '/home/sasank/Documents/GitRepos/Sasank_JTML_seg/data/3_2_22_fem/val_3_2_22_fem.csv',
            'test_data': '/home/sasank/Documents/GitRepos/Sasank_JTML_seg/data/3_2_22_fem/test_3_2_22_fem.csv'
        }
        """
        self.init = {
            'PROJECT_NAME': 'Keypoint Estimation',
            'MODEL_NAME': 'Tib_64KP',
            'RUN_NAME': time.strftime('%Y-%m-%d-%H-%M-%S'),
            'WANDB_RUN_GROUP': 'Local',
            'FAST_DEV_RUN': False,  # Runs inputted batches (True->1) and disables logging and some callbacks
            'MAX_EPOCHS': 10,
            'MAX_STEPS': -1,    # -1 means it will do all steps and be limited by epochs
            'STRATEGY': None    # This is the training strategy. Should be 'ddp' for multi-GPU (like HPG)
        }
        self.etl = {
            'RAW_DATA_FILE': -1,
            'DATA_DIR': "data",
            # Lol what is this?
            'KEYPOINT_DIRECTORY': "keypoints",
            'KEYPOINT_TXT_FILES': ['tib_KPlabels_16.txt'],
            'VAL_SIZE':  0.2,       # looks sus
            'TEST_SIZE': 0.01,      # I'm not sure these two mean what we think
            #'random_state': np.random.randint(1,50)
            # HHG2TG lol; deterministic to aid reproducibility
            'RANDOM_STATE': 42,

            'CUSTOM_TEST_SET': False,
            'TEST_SET_NAME': '/my/test/set.csv'
        }

        self.dataset = {
            'DATA_NAME': 'Ten_Dogs_64KP',
            'SUBSET_PIXELS': True,
            'IMAGE_HEIGHT': 1024,
            'IMAGE_WIDTH': 1024,
            'MODEL_TYPE': 'tib',        # how should we do this? not clear this is still best...
            'CLASS_LABELS': {0: 'bone', 1: 'background'},
            'NUM_KEY_POINTS': 64,
            'IMG_CHANNELS': 1,      # Is this different from self.module['NUM_IMAGE_CHANNELS']
            'STORE_DATA_RAM': False,
            'IMAGE_THRESHOLD': 0,
            'USE_ALBUMENTATIONS': False,

            # What do these do?
            'NUM_PRINT_IMG' : 1,
            'KP_PLOT_RAD' : 3,

            #'NUM_POINTS' : 128,

            'GAUSSIAN_STDDEV' : 5,
            'GAUSSIAN_AMP' : 1e3,

            'STORE_DATA_RAM' : False,

            'CROP_IMAGES' : False,
            'CROP_MIN_X' : 0.29,
            'CROP_MAX_X' : 0.84,
            'CROP_MIN_Y' : 0.45,
            'CROP_MAX_Y' : 0.95,
            
            'IMAGES_PER_GRID': 1,
            'per_grid_image_count_height' : 1, 
            'per_grid_image_count_width' : 1
        }

        """
        # segmentation_net_module needs to be below dataset because it uses dataset['IMG_CHANNELS']
        self.keypoint_net_module = {
            'NUM_KEY_POINTS': 128,
            'NUM_IMG_CHANNELS': self.dataset['IMG_CHANNELS']
        }
        """

        self.datamodule = {
            'IMAGE_DIRECTORY': '/media/sasank/LinuxStorage/Dropbox (UFL)/Canine Kinematics Data/TPLO_Ten_Dogs_grids/',
            'CKPT_FILE': None,
            'USE_NAIVE_TEST_SET': False,
            'BATCH_SIZE': 2,
            'SHUFFLE': True,        # Only for training; for test and val this is set in the datamodule script to False
            'NUM_WORKERS': 2,
            'PIN_MEMORY': False
            #'SUBSET_PIXELS': True - this is now in dataset
        }


        # hyperparameters for training
        self.hparams = {
            'LOAD_FROM_CHECKPOINT': False,
            'learning_rate': 1e-3
        }

        #self.transform = None
        self.transform = \
        A.Compose([
            # Let's do only rigid transformations for now
            A.HorizontalFlip(p=0.9),
            A.VerticalFlip(p=0.9),
            A.RandomRotate90(p=0.9),
            A.Transpose(p=0.9),
            #A.RandomGamma(always_apply=False, p = 0.5,gamma_limit=(10,300)),
            #A.ShiftScaleRotate(always_apply = False, p = 0.5,shift_limit=(-0.06, 0.06), scale_limit=(-0.1, 0.1), rotate_limit=(-180,180), interpolation=0, border_mode=0, value=(0, 0, 0)),
            #A.Blur(always_apply=False, blur_limit=(3, 10), p=0.2),
            #A.Flip(always_apply=False, p=0.5),
            #A.InvertImg(always_apply=False, p=0.5),
            #A.MultiplicativeNoise(always_apply=False, p=0.25, multiplier=(0.1, 2), per_channel=True, elementwise=True)
            #A.ElasticTransform(always_apply=False, p=0.85, alpha=0.5, sigma=150, alpha_affine=50.0, interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None, approximate=False),
            #A.CoarseDropout(always_apply = False, p = 0.25, min_holes = 1, max_holes = 100, min_height = 25, max_height=25),
        ],
        p=0.85)

In [5]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from skimage import io
import cv2

import pytorch_lightning as pl
import wandb

from loss import kp_loss



class KeypointDataset(torch.utils.data.Dataset):

    def __init__(self, config, evaluation_type, transform=None):
        """
        Args:
            config (config): Dictionary of vital constants about data.
            store_data_ram (boolean): Taken from config.
            evaluation_type (string): Dataset evaluation type (must be 'training', 'validation', 'test', or 'naive')
            num_points (int): Taken from config.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        # Create local copies of the arguments
        self.config = config
        self.num_points = self.config.dataset['NUM_KEY_POINTS']
        self.transform = self.config.transform
        
        # Check that evaluation_type is valid and then store
        if evaluation_type in ['train', 'val', 'test', 'naive']:
            self.evaluation_type = evaluation_type
        else:
            raise Exception('Incorrect evaluation type! Must be either \'train\', \'val\', \'test\', or \'naive\'.')

        # Load the data from the big_data CSV file into a pandas dataframe
        #self.data = pd.read_csv(os.path.join(self.config.etl['DATA_DIR'], self.config.dataset['DATA_NAME'], self.evaluation_type + '_' + self.config.dataset['DATA_NAME'] + '.csv'))
        self.data = pd.read_csv('/home/sasank/Documents/GitRepos/Stifle-Keypoints/data/Ten_Dogs_64KP/test_Ten_Dogs_64KP.csv')
        # Print number of rows in the dataframe
        print('Number of rows in the dataframe: ', len(self.data))
        # Print 'Image address' of the first row
        print('Image address of the first row: ', self.data.iloc[0]['Image address'])
        # Print 'Image address' of the last row
        print('Image address of the last row: ', self.data.iloc[-1]['Image address'])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        #idx += 1    # Add 1 because the first row is the column names

        # Get the row of the dataframe
        row = self.data.iloc[idx]

        # Get the image name
        image_name = row['Image address']

        # Get the image
        image = io.imread(os.path.join(self.config.datamodule['IMAGE_DIRECTORY'], image_name))
        full_image = image             # Save full image (no subset_pixels) for visualization

        # Get the keypoint labels and segmentation labels
        if self.config.dataset['MODEL_TYPE'] == 'fem':
            kp_label = row['Femur 2D KP points']
            seg_label = io.imread(os.path.join(self.config.datamodule['IMAGE_DIRECTORY'], row['Fem label address']))
        elif self.config.dataset['MODEL_TYPE'] == 'tib':
            kp_label = row['Tibia 2D KP points']
            seg_label = io.imread(os.path.join(self.config.datamodule['IMAGE_DIRECTORY'], row['Tib label address']))
        else:
            raise Exception('Incorrect model type! Must be either \'fem\' or \'tib\'.')

        kp_label = kp_label[2:-2]
        kp_label = kp_label.split(']\n [')
        kp_label = [np.array([float(x) for x in list(filter(None, kp.split(' ')))]) for kp in kp_label]
        kp_label = np.array(kp_label)
        
        # * Subset Pixels
        if self.config.dataset['SUBSET_PIXELS'] == True:
            label_dst = np.zeros_like(seg_label)
            label_normed = cv2.normalize(seg_label, label_dst, alpha = 0, beta = 1, norm_type = cv2.NORM_MINMAX)
            seg_label = label_normed

            kernel = np.ones((30,30), np.uint8)
            label_dilated = cv2.dilate(seg_label, kernel, iterations = 5)
            image_subsetted = cv2.multiply(label_dilated, image)
            image = image_subsetted

        image = torch.FloatTensor(image[None, :, :]) # Store as byte (to save space) then convert when called in __getitem__
        full_image = torch.FloatTensor(full_image[None, :, :]) # Store as byte (to save space) then convert when called in __getitem__
        seg_label = torch.FloatTensor(seg_label[None, :, :])
        #kp_label = torch.FloatTensor(kp_label.reshape(-1))      # Reshape to 1D array so that it's 2*num_keypoints long
        kp_label = torch.FloatTensor(kp_label)          # kp_label is of shape (num_keypoints, 2)
        assert kp_label.shape == (self.num_points, 2), "Keypoint label shape is incorrect!"
        #print("kp_label.shape:")
        #print(kp_label.shape)


    
        # Form sample and transform if necessary
        sample = {'image': image,
                    'img_name': image_name,
                    'kp_label': kp_label,
                    'seg_label': seg_label,
                    'full_image': full_image}
        assert self.transform is not None, "Transforms not implemented yet!"
        if self.transform and self.config.dataset['USE_ALBUMENTATIONS'] == True:
            sample = self.transform(sample)     # TODO: Implement transforms. Seems like we'll have to reshape the keypoints to be num_keypoints x 2 instead of 2*num_keypoints x 1
        return sample

In [6]:
fem_config = Fem_Configuration()
tib_config = Tib_Configuration()
fem_dataset = KeypointDataset(fem_config, 'test')
tib_dataset = KeypointDataset(tib_config, 'test')

Number of rows in the dataframe:  365
Image address of the first row:  grid_Calib file test_000000001769.tif
Image address of the last row:  grid_Calib file test_000000000452.tif
Number of rows in the dataframe:  365
Image address of the first row:  grid_Calib file test_000000001769.tif
Image address of the last row:  grid_Calib file test_000000000452.tif


In [9]:
#from lit_KPResNet import KeypointNetModule
from torch.utils.data import DataLoader

# later on, will need to change to accept outputs from updated model (compatible with Albumentations), where the kps have been multiplied by 1024 and the y reversed (by 1024 - y)

FEM_CHECKPOINT_PATH = '/home/sasank/Documents/GitRepos/Stifle-Keypoints/checkpoints/Fem64_200Epochs_Resnet152.ckpt'
TIB_CHECKPOINT_PATH = '/home/sasank/Documents/GitRepos/Stifle-Keypoints/checkpoints/Tib64_200Epochs_Resnet152.ckpt'
fem_model = KeypointNetModule.load_from_checkpoint(FEM_CHECKPOINT_PATH, config=fem_config, wandb_run = None)
tib_model = KeypointNetModule.load_from_checkpoint(TIB_CHECKPOINT_PATH, config=tib_config, wandb_run = None)

# ! This csv_file does not actually get used to get the outputs. It is only used to get the number of entries in the csv file.
# ! Edit the csv file in the Dataset object above to change the csv file used to get the outputs.
csv_file = pd.read_csv('/home/sasank/Documents/GitRepos/PnP-Solver/kp_estimates/test_set_64KP.csv', index_col=0)

# Number of entries in fem_dataset
print('Number of entries in fem_dataset is :')
print(len(fem_dataset))

print('Number of data rows in the csv file is :')
print(len(csv_file))

fem_dataloader = DataLoader(fem_dataset, batch_size=1, shuffle=False, num_workers=0)
tib_dataloader = DataLoader(tib_dataset, batch_size=1, shuffle=False, num_workers=0)

# Iterate through the dataloader and get the outputs

print('Number of batches in the dataloader is :')
print(len(fem_dataloader))

np.set_printoptions(suppress=True)      # Don't use scientific notation

####### ONNX EXPORT #######
#"""
fem_model.eval()
fem_model.to('cpu')
fem_model = fem_model.cpu()
dummy_input = torch.randn(1, 1, 1024, 1024)
torch.onnx.export(fem_model, dummy_input, "Fem64_200Epochs_R152_nonverbose.onnx", verbose=False)

tib_model.eval()
tib_model.to('cpu')
tib_model = tib_model.cpu()
dummy_input = torch.randn(1, 1, 1024, 1024)
torch.onnx.export(tib_model, dummy_input, "Tib64_200Epochs_R152_nonverbose.onnx", verbose=False)

sys.exit(0)
#"""
###########################

# Femur
for i, sample in enumerate(fem_dataloader):
    image = sample['image']
    img_name = sample['img_name'][0]
    #print('image name is ' + img_name)
    #print('image name is ' + img_name[0])
    #image.unsqueeze_(0)
    output = fem_model(image)

    # We will plot the outputs, which are 2D keypoints, to self.csv_file in the row that corresponds to the image name
    keypoints = output.detach().cpu().numpy()
    # Make the keypoints a 2D array of shape (num_keypoints, 2)
    keypoints = keypoints.reshape(fem_config.dataset['NUM_KEY_POINTS'], 2)

    # Find the row that corresponds to the image name and put the keypoints there
    csv_file.loc[csv_file['Image address'] == img_name, 'Femur PR KP points'] = str(keypoints)

# Tibia
for i, sample in enumerate(tib_dataloader):
    image = sample['image']
    img_name = sample['img_name'][0]
    #image.unsqueeze_(0)
    output = tib_model(image)

    # We will plot the outputs, which are 2D keypoints, to self.csv_file in the row that corresponds to the image name
    keypoints = output.detach().cpu().numpy()
    # Make the keypoints a 2D array of shape (num_keypoints, 2)
    keypoints = keypoints.reshape(tib_config.dataset['NUM_KEY_POINTS'], 2)

    # Find the row that corresponds to the image name and put the keypoints there
    csv_file.loc[csv_file['Image address'] == img_name, 'Tibia PR KP points'] = str(keypoints)

csv_file.to_csv('/home/sasank/Documents/GitRepos/PnP-Solver/kp_estimates/test_predicted_R152_asdf.csv')

Lightning automatically upgraded your loaded checkpoint from v1.7.6 to v1.9.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../checkpoints/Fem64_200Epochs_Resnet152.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.7.6 to v1.9.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../checkpoints/Tib64_200Epochs_Resnet152.ckpt`


Number of entries in fem_dataset is :
365
Number of data rows in the csv file is :
365
Number of batches in the dataloader is :
365


SystemExit: 0

/home/sasank/miniconda3/envs/jtml/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
data = pd.read_csv('/home/sasank/Documents/GitRepos/Stifle-Keypoints/data/personal_val.csv', index_col=0)

# Check if there are rows with Patient number = 3 and Session number = 1
print(data.loc[(data['Patient number'] == 3) & (data['Session number'] == 1)])

# Print unique values of Session number for Patient number = 3
print(data.loc[data['Patient number'] == 3]['Session number'].unique())

Empty DataFrame
Columns: [Image address, Fem label address, Tib label address, Patient name, Patient number, Session name, Session number, Movement name, Movement number, Principal distance, X offset, Y offset, Scale, Calib translation, Calib y_hat, Calib z_hat, Femur STL, Tibia STL, Femur kinematics (x_tran, y_tran, z_tran, z_rot, x_rot, y_rot), Tibia kinematics (x_tran, y_tran, z_tran, z_rot, x_rot, y_rot), Femur 3D KP points, Tibia 3D KP points, Femur 2D KP points, Tibia 2D KP points]
Index: []

[0 rows x 24 columns]
[2 3]


In [ ]:
PRINT_DIR = '/media/sasank/LinuxStorage/Dropbox (UFL)/Canine Kinematics Data/image_check/pruned_tib/'
# Iterate through the dataset and print the shapes of the images and labels
for i in range(len(dataset)):
    print("image number: " + str(i))
    batch = dataset[i]
    images = batch['image']
    kp_labels = batch['kp_label']
    img_names = batch['img_name']
    title = ''




    num_images = images.shape[0]
    num_keypoints = kp_labels.shape[0]
    assert num_keypoints == 64, "Number of keypoints is incorrect!"
    images = images.cpu()
    kp_labels = kp_labels.cpu()
    kp_labels = kp_labels.numpy()

    output_image_vector = []

    for i in range(0, num_images):
        fig, ax = plt.subplots(1, 1, figsize=(10, 10), squeeze=False)

        # ! TODO: Is this the right way to do this? Is there something wrong here with offsets or something?
        #kp_labels[i][:, 0] = +1 * kp_labels[i][:, 0] * 1024
        kp_labels[:, 0] = +1 * kp_labels[:, 0] * 1024
        #kp_labels[i][:, 1] = -1 * kp_labels[i][:, 1] * 1024 + 1024
        kp_labels[:, 1] = -1 * kp_labels[:, 1] * 1024 + 1024
        # Do some stuff so that img is shown correctly
        img = images.numpy()
        img = np.transpose(img, (1, 2, 0))  # Transpose the output so that it's the same way as img
        img = np.dstack((img, img, img))    # Make it 3 channels
        ax[0][0].imshow((img * 255).astype(np.uint8))  # The multiplying by 255 and stuff is so it doesn't get clipped or something

        for j in range(num_keypoints):
            #ax[0][0].text(labels[i][j, 0], labels[i][j, 1], str(j), color='m')        # Silenced this for now since we have 64 keypoints
            ax[0][0].plot(kp_labels[j, 0], kp_labels[j, 1], color='orange', marker='.', markersize=5)
            #ax[0][0].plot([kp_labels[i][j, 0], kp_preds[i][j, 0]], [labels[i][j, 1], preds[i][j, 1]], color='limegreen', linestyle='-')
        image_name = img_names.split('/')[-1]    # Format img_names[i] so that only the part after the last '/' is shown
        ax[0][0].set_title(title + ' {}'.format(image_name))
        fig.savefig(os.path.join(PRINT_DIR, image_name))
        output_image_vector.append(fig)
        plt.close()
    

NameError: name 'dataset' is not defined